# Set up Azure SQL Database for catching the bus application

This is a SQL Notebook, which allows you to separate text and code blocks and save code results. Azure Data Studio supports several languages, referred to as kernels, including SQL, PowerShell, Python, and more.

In this activity, you'll learn how to import data into Azure SQL Database and create tables to store the route data, geofence data, and real-time bus information.

## Connect to `bus-db`

At the top of the window, select **Select Connection** \> **Change Connection** next to "Attach to".

Under _Recent Connections_ select your `bus-db` connection.

You should now see it listed next to _Attach to_.

## Part 1: Import the bus route data from Azure Blob Storage

The first step in configuring the database for the scenario is to import a CSV file that contains route information data. The following script will walk you through that process. Full documentation on "Accessing data in a CSV file referencing an Azure blob storage location" here: [https://docs.microsoft.com/sql/relational-databases/import-export/examples-of-bulk-access-to-data-in-azure-blob-storage](https://docs.microsoft.com/sql/relational-databases/import-export/examples-of-bulk-access-to-data-in-azure-blob-storage).

You need to first create a table and schema for data to be loaded into.

In [1]:
CREATE TABLE [dbo].[Routes]
(
	[Id] [int] NOT NULL,
	[AgencyId] [varchar](100) NULL,
	[ShortName] [varchar](100) NULL,
	[Description] [varchar](1000) NULL,
	[Type] [int] NULL
)
GO
ALTER TABLE [dbo].[Routes] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
)
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.429

The next step is to create a master key.

In [2]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'mySuperStr0n9P@assw0rd!'

Commands completed successfully.

Total execution time: 00:00:00.168

A master key is required to create a `DATABASE SCOPED CREDENTIAL` value because Blob storage is not configured to allow public (anonymous) access. The credential refers to the Blob storage account, and the data portion specifies the container for the store return data.

We use a shared access signature as the identity that Azure SQL knows how to interpret. The secret is the SAS token that you can generate from the Blob storage account. In this example, the SAS token for a storage account that you don't have access to is provided so you can access only the store return data.

In [4]:
DROP EXTERNAL DATA SOURCE RouteData
DROP DATABASE SCOPED CREDENTIAL AzureBlobCredentials

CREATE DATABASE SCOPED CREDENTIAL AzureBlobCredentials
WITH IDENTITY = 'SHARED ACCESS SIGNATURE',
SECRET = 'sp=r&st=2021-08-18T11:11:04Z&se=2024-06-30T19:11:04Z&spr=https&sv=2020-08-04&sr=c&sig=PytNI7Ypncp0XuDjsTM1pRjmbZQK3zxYbDumbllXtWg%3D' -- Omit any leading question mark


Commands completed successfully.

Total execution time: 00:00:00.188

Next, create an external data source to the container.

In [5]:
CREATE EXTERNAL DATA SOURCE RouteData
WITH (
	TYPE = blob_storage,
	LOCATION = 'https://azuresqlworkshopsa01.blob.core.windows.net/bus',
	CREDENTIAL = AzureBlobCredentials
)

Commands completed successfully.

Total execution time: 00:00:00.166

Now you are ready to bring in the data.

In [5]:
DELETE FROM dbo.[Routes];
INSERT INTO dbo.[Routes]
	([Id], [AgencyId], [ShortName], [Description], [Type])
SELECT 
	[Id], [AgencyId], [ShortName], [Description], [Type]
FROM 
OPENROWSET
	( 
		BULk 'routes.txt', 
		DATA_SOURCE = 'RouteData', 
		FORMATFILE = 'routes.fmt', 
		FORMATFILE_DATA_SOURCE = 'RouteData', 
		FIRSTROW=2,
		FORMAT='csv'
	) t;

(0 rows affected)

(132 rows affected)

Total execution time: 00:00:02.543

Finally, let's look at what's been inserted relative to the route we'll be tracking.

In [6]:
SELECT * FROM dbo.[Routes] WHERE [Description] LIKE '%Education Hill%'

(132 rows affected)

Total execution time: 00:00:00.186

Id,AgencyId,ShortName,Description,Type
100001,1,1,Kinnear - Downtown Seattle,3
100002,1,10,Capitol Hill - Downtown Seattle,3
100003,1,101,Renton Transit Center - Downtown Seattle,3
100004,1,105,Renton Highlands - Renton Transit Center,3
100005,1,106,Renton Transit Center - Skyway - Downtown Seattle,3
100006,1,107,Renton Transit Center - Rainier Beach,3
100009,1,11,Madison Park - Capitol Hill - Downtown Seattle,3
100011,1,111,Lake Kathleen - Downtown Seattle,3
100016,1,118,Tahlequah - Vashon - Downtown Seattle,3
100017,1,119,Dockton - Vashon - Downtown Seattle,3


## Part 2: Create necessary tables

### Select a route to monitor

Now that you've added the route information, you can select the route to be a "Monitored Route". This will come in handy if you later choose to monitor multiple routes. For now, you will just add the "Education Hill - Crossroads - Eastgate" route.

In [7]:
-- Create MonitoredRoutes table
CREATE TABLE [dbo].[MonitoredRoutes]
(
	[RouteId] [int] NOT NULL
)
GO
ALTER TABLE [dbo].[MonitoredRoutes] ADD PRIMARY KEY CLUSTERED 
(
	[RouteId] ASC
)
GO
ALTER TABLE [dbo].[MonitoredRoutes] 
ADD CONSTRAINT [FK__MonitoredRoutes__Router] 
FOREIGN KEY ([RouteId]) REFERENCES [dbo].[Routes] ([Id])
GO

-- Monitor the "Education Hill - Crossroads - Eastgate" route
INSERT INTO dbo.[MonitoredRoutes] (RouteId) VALUES (100113);

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.679

### Select a GeoFence to monitor

In addition to monitoring specific bus routes, you will want to monitor certain GeoFences so you can ultimately get notified when your bus enters or exits where you are (i.e. the GeoFence). For now, you will add a small GeoFence that represents the area near the "Crossroads" bus stop.

In [8]:
-- Create GeoFences table
CREATE SEQUENCE [dbo].[global]
    AS INT
    START WITH 1
    INCREMENT BY 1
GO
CREATE TABLE [dbo].[GeoFences](
	[Id] [int] NOT NULL,
	[Name] [nvarchar](100) NOT NULL,
	[GeoFence] [geography] NOT NULL
) 
GO
ALTER TABLE [dbo].[GeoFences] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
)
GO
ALTER TABLE [dbo].[GeoFences] ADD DEFAULT (NEXT VALUE FOR [dbo].[global]) FOR [Id]
GO
CREATE SPATIAL INDEX [ixsp] ON [dbo].[GeoFences]
(
	[GeoFence]
) USING GEOGRAPHY_AUTO_GRID 
GO

-- Create a GeoFence
INSERT INTO dbo.[GeoFences] 
	([Name], [GeoFence]) 
VALUES
	('Crossroads', 0xE6100000010407000000B4A78EA822CF4740E8D7539530895EC03837D51CEACE4740E80BFBE630895EC0ECD7DF53EACE4740E81B2C50F0885EC020389F0D03CF4740E99BD2A1F0885EC00CB8BEB203CF4740E9DB04FC23895EC068C132B920CF4740E9DB04FC23895EC0B4A78EA822CF4740E8D7539530895EC001000000020000000001000000FFFFFFFF0000000003);
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:01.309

### Create table to track activity in the GeoFence

Next, create a system-versioned table to keep track of what activity is currently happening within the GeoFence. This means tracking buses entering, exiting, and staying within a given GeoFence. Another table within that table will serve as a histroical log for all activity.

In [9]:
CREATE TABLE [dbo].[GeoFencesActive] 
(
	[Id] [int] IDENTITY(1,1) NOT NULL PRIMARY KEY CLUSTERED,
	[VehicleId] [int] NOT NULL,
	[DirectionId] [int] NOT NULL,
	[GeoFenceId] [int] NOT NULL,
	[SysStartTime] [datetime2](7) GENERATED ALWAYS AS ROW START NOT NULL,
	[SysEndTime] [datetime2](7) GENERATED ALWAYS AS ROW END NOT NULL,
	PERIOD FOR SYSTEM_TIME ([SysStartTime], [SysEndTime])
)
WITH ( SYSTEM_VERSIONING = ON ( HISTORY_TABLE = [dbo].[GeoFencesActiveHistory] ) )
GO

Commands completed successfully.

Total execution time: 00:00:00.179

### Create a table to store real-time bus data

You'll need one last table to store the real-time bus data as it comes in.

In [10]:
CREATE TABLE [dbo].[BusData](
	[Id] [int] IDENTITY(1,1) NOT NULL,
	[DirectionId] [int] NOT NULL,
	[RouteId] [int] NOT NULL,
	[VehicleId] [int] NOT NULL,
	[Location] [geography] NOT NULL,
	[TimestampUTC] [datetime2](7) NOT NULL,
	[ReceivedAtUTC] [datetime2](7) NOT NULL
)
GO
ALTER TABLE [dbo].[BusData] ADD DEFAULT (SYSUTCDATETIME()) FOR [ReceivedAtUTC]
GO
ALTER TABLE [dbo].[BusData] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
) 
GO
CREATE NONCLUSTERED INDEX [ix1] ON [dbo].[BusData]
(
	[ReceivedAtUTC] DESC
) 
GO
CREATE SPATIAL INDEX [ixsp] ON [dbo].[BusData]
(
	[Location]
) USING GEOGRAPHY_AUTO_GRID 
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.833

Confirm you've created the tables with the following.

In [12]:
SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_SCHEMA = 'dbo'

(6 rows affected)

Total execution time: 00:00:00.172

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
bus-db,dbo,Routes,BASE TABLE
bus-db,dbo,MonitoredRoutes,BASE TABLE
bus-db,dbo,GeoFences,BASE TABLE
bus-db,dbo,GeoFencesActive,BASE TABLE
bus-db,dbo,GeoFencesActiveHistory,BASE TABLE
bus-db,dbo,BusData,BASE TABLE
